In [1]:
# Deepdream refactored to work with my models and code
import os
# Set path to root directory so we can access core, tasks, data, etc.
os.chdir('..')

In [2]:
from core.utils.utils import *

In [12]:
import tensorflow as tf

def load_model(sess, params):
    """Load model from checkpoint"""
    # First load graph
    # .meta file defines graph - they should all be the same? So just take any one
    # meta_file = [f for f in os.listdir(params['ckpt_dirpath']) if f.endswith('meta')][0]
    # meta_filepath = os.path.join(params['ckpt_dirpath'], meta_file)
    # saver = tf.train.import_meta_graph(meta_filepath)

    # Load weights
    saver = None
#     saver = tf.train.Saver(tf.global_variables())
    if params['load_epoch'] is not None:        # load the checkpoint for the given epoch
        ckpt_name = _get_ckpt_basename(params) + '-' + str(params['load_epoch'])
        ckpt_path = os.path.join(params['ckpt_dirpath'], ckpt_name)
        # saver.restore(sess, os.path.join(params['ckpt_dirpath'], ckpt_name))
    else:       # 'checkpoint' binary file keeps track of latest checkpoints. Use it to to get most recent
        ckpt = tf.train.get_checkpoint_state(params['ckpt_dirpath'])
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        ckpt_path = os.path.join(params['ckpt_dirpath'], ckpt_name)
#     saver.restore(sess, ckpt_path)

    return saver, ckpt_path

def freeze_graph(ckpt_dirpath, arch, obj, load_epoch=None):
    params = {'ckpt_dirpath': ckpt_dirpath, 'arch': arch, 'obj': obj, 'load_epoch': load_epoch}

    saver, ckpt_path = load_model(None, params)
    input_checkpoint = ckpt_path
    
    # We retrieve our checkpoint fullpath
    # checkpoint = tf.train.get_checkpoint_state(model_folder)
    # input_checkpoint = checkpoint.model_checkpoint_path
    #
    # # We precise the file fullname of our freezed graph
    # absolute_model_folder = "/".join(input_checkpoint.split('/')[:-1])
#     output_graph = absolute_model_folder + "/frozen_model.pb"
    output_graph = os.path.join(ckpt_dirpath, "frozen_model.pb")

    #
#     Before exporting our graph, we need to precise what is our output node
#     This is how TF decides what part of the Graph he has to keep and what part it can dump
#     NOTE: this variables is plural, because you can have multiple output nodes
    # output_node_names = "Accuracy/predictions"
    output_node_names = "img_batch,fc/4_w,fc/4_b"
    
    # We clear the devices, to allow TensorFlow to control on the loading where it wants operations to be calculated
    clear_devices = True
    
    # We import the meta graph and retrive a Saver
    saver = tf.train.import_meta_graph(input_checkpoint + '.meta', clear_devices=clear_devices)

    # We retrieve the protobuf graph definition
    graph = tf.get_default_graph()
    input_graph_def = graph.as_graph_def()

    # We start a session and restore the graph weights
    with tf.Session() as sess:
        print input_checkpoint
        saver.restore(sess, input_checkpoint)

        # We use a built-in TF helper to export variables to constant
        output_graph_def = graph_util.convert_variables_to_constants(
            sess, # The session is used to retrieve the weights
            input_graph_def, # The graph_def is used to retrieve the nodes
            output_node_names.split(",") # The output node names are used to select the usefull nodes
        )

        # Finally we serialize and dump the output graph to the filesystem
        with tf.gfile.GFile(output_graph, "wb") as f:
            f.write(output_graph_def.SerializeToString())
        print("%d ops in the final graph." % len(output_graph_def.node))
        
def _get_ckpt_basename(params):
    """Use to save and load"""
    return params['arch'] + '-' + params['obj']


In [13]:
freeze_graph('tasks/image_sent/checkpoints/2016-12-26___08-45-18/', 'basic_cnn', 'sent_biclass', load_epoch=4)

tasks/image_sent/checkpoints/2016-12-26___08-45-18/basic_cnn-sent_biclass-4
INFO:tensorflow:Froze 24 variables.


INFO:tensorflow:Froze 24 variables.


AssertionError: img_batch:0 is not in graph

In [11]:
import numpy as np
model_fn = 'tasks/image_sent/checkpoints/2016-12-26___08-45-18/frozen_model.pb'


# creating TensorFlow session and loading the model
graph = tf.Graph()
sess = tf.InteractiveSession(graph=graph)
with tf.gfile.FastGFile(model_fn, 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
t_input = tf.placeholder(np.float32, name='img_batch') # define the input tensor
imagenet_mean = 117.0
t_preprocessed = tf.expand_dims(t_input-imagenet_mean, 0)
tf.import_graph_def(graph_def, {'img_batch':t_preprocessed})

ValueError: Attempted to map inputs that were not found in graph_def: [img_batch:0]